In [3]:
import pandas as pd
from sqlalchemy import create_engine
password = 'sqL7910253'

In [4]:
# Read in business data from JSON, into a Dataframe
yelp_json = '../yelp_ETL_data/business.json'
yelp_df = pd.read_json(yelp_json, lines=True)
yelp_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [5]:
# Flatten dictionaries in 'attributes' column into separate columns
attributes_df = yelp_df['attributes'].apply(pd.Series)
attributes_df.head()

,GoodForKids,RestaurantsReservations,GoodForMeal,BusinessParking,Caters,NoiseLevel,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,...,BYOBCorkage,DriveThru,Smoking,AgesAllowed,HairSpecializesIn,Corkage,BYOB,DietaryRestrictions,Open24Hours,RestaurantsCounterService
0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",True,u'loud',True,True,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",False,u'average',True,True,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Pull just the 'Caters' column
caters_df = attributes_df['Caters']
caters_df.head()

0      NaN
1     True
2    False
3      NaN
4      NaN
Name: Caters, dtype: object

In [12]:
# Concatenate 'Caters' column to original DF along column axis, and drop original 'attributes' column
cleaned_df = pd.concat([yelp_df.drop(['attributes'], axis=1), caters_df], axis=1)

# Select only the necessary columns and rename
cleaned_df = cleaned_df[['business_id', 'name', 'address', 'state', 'postal_code', 'Caters', 'stars', 'review_count']]
cleaned_df.rename(columns={'postal_code':'zip_code','Caters':'has_catering'}, inplace=True)
cleaned_df.head()

,business_id,name,address,state,zip_code,has_catering,stars,review_count
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,AZ,85016,NaN,3.0,5
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,ON,L5R 3E7,True,2.5,128
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",NC,28210,False,4.0,170
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",AZ,85338,NaN,5.0,3
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",NC,28217,NaN,4.0,4


In [13]:
# Trim DF to only include rows with True or False in 'has_catering' column
businesses_df = cleaned_df.loc[(cleaned_df['has_catering'] == 'True') | (cleaned_df['has_catering'] == 'False')]
businesses_df.head()

,business_id,name,address,state,zip_code,has_catering,stars,review_count
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,ON,L5R 3E7,True,2.5,128
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",NC,28210,False,4.0,170
17,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",NV,89119,True,4.0,40
25,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,NV,89156,False,4.5,184
29,NDuUMJfrWk52RA-H-OtrpA,Bolt Fresh Bar,1170 Queen Street W,ON,M6J 1J5,False,3.0,57


In [14]:
# Read in income data from CSV, into a DataFrame
income_csv = '../yelp_ETL_data/MedianZIP.csv'
income_df = pd.read_csv(income_csv)

In [15]:
# Rename columns to match business data (to match keys in SQL DB)
income_df = income_df[['Zip','Median']].rename(columns={'Zip':'zip_code','Median':'median_income'})
income_df.head()

,zip_code,median_income
0,1001,"56,663"
1,1002,"49,853"
2,1003,"28,462"
3,1005,"75,423"
4,1007,"79,076"


In [16]:
connection_string = f"root:{password}@localhost/yelp_delivery_db"
engine = create_engine(f"mysql://{connection_string}")

In [17]:
engine.table_names()

['businesses', 'median_income']

In [18]:
businesses_df.to_sql(name='businesses', con=engine, if_exists='replace', index=False)

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 7-8: ordinal not in range(256)

In [ ]:
income_df.to_sql(name='median_income', con=engine, if_exists='replace', index=False)